In [181]:
import socket
import sys
import platform
import os
from scapy.all import sniff, wrpcap
from config import constants

In [182]:
CATEGORIES = constants.CATEGORIES
print(CATEGORIES)

['cloud_service', 'e-mail', 'games', 'information', 'music', 'social_network', 'video_hosting']


In [183]:
DOMAINS = {
    'music': ["spotify.com", "open.spotify.com", "music.yandex.ru", "mc.yandex.ru", "music.apple.com", "zaycev.net"],
    'social_network': ["vk.com", "ok.ru", "web.telegram.org", "telegram.org", "t.me"],
    'video_hosting': ["twitch.tv", "youtube.com", "rutube.ru", "vkvideo.ru", "kinopoisk.ru", "hd.kinopoisk.ru", "ivi.ru", "premier.one"],
    'cloud_service': ["disk.yandex.ru", "cloud.mail.ru", "drive.google.com", "onedrive.live.com"],
    'e-mail': ["mail.google.com", "mail.yandex.ru", "mail.ru", "outlook.live.com", "gmail.com", "vk.mail.ru"],
    'information': ["wikipedia.org", "nstu.ru", "ngs.ru", "gismeteo.ru", "habr.com", "tass.ru", "hse.ru"]
}

In [184]:
CURRENT_CATEGORY = 'music'
DURATION = 300

In [185]:
print(f"Сбор трафика для {CURRENT_CATEGORY} с продолжительностью {DURATION}c.")

Сбор трафика для music с продолжительностью 300c.


In [186]:
system = platform.system().lower()
print(f"Текущая ОС {system}")

Текущая ОС darwin


In [187]:
if system == "windows":
    INTERFACE = "Беспроводная сеть"
else:
    INTERFACE = "en0"

print(INTERFACE)

en0


In [188]:
def resolve_domains(domains):
    ip_list = []
    for domain in domains:
        try:
            addr_info = socket.getaddrinfo(
                domain,
                None,
                family=0,
                type=socket.SOCK_STREAM,
                flags=socket.AI_ALL | socket.AI_V4MAPPED
            )

            ip_set = set()
            for info in addr_info:
                ip = info[4][0]
                ip_set.add(ip)
            ip_list.extend(ip_set)

            print(f"Домен {domain}: {list(ip_set)}")
        except socket.gaierror:
            return []

    return list(set(ip_list))

In [189]:
def create_bpf_filter():
    ip_list = resolve_domains(DOMAINS.get(CURRENT_CATEGORY))
    if not ip_list:
        return ""

    ip_filter = " or ".join(f"host {ip}" for ip in ip_list)

    return f"{ip_filter}"

In [190]:
def start_capture(interface, output_file, duration=60):
    packets = []

    bpf_filter = create_bpf_filter()

    print(bpf_filter)

    if not bpf_filter:
        print("Не удалось создать фильтр для захвата трафика")
        sys.exit(1)

    print(f"BPF-фильтр: {bpf_filter}")
    print(f"Начало захвата трафика на интерфейсе {interface}")

    def packet_callback(packet):
        packets.append(packet)
        print(f"Захвачен пакет: {packet.summary()}")

    try:
        sniff(iface=interface, prn=packet_callback, timeout=duration, filter=bpf_filter)
        print(f"Сохранение {len(packets)} пакетов в файл {output_file}...")
        wrpcap(output_file, packets)
        print(f"Захват завершен. Файл сохранен: {output_file}")

    except Exception as e:
        print(f"Ошибка при захвате трафика: {e}")
        sys.exit(1)

In [191]:
RAW_PATH = f'raw/{CURRENT_CATEGORY}'
pcap_files = [f for f in os.listdir(RAW_PATH) if f.endswith('.pcap')]
FILENAME = f"{len(pcap_files) + 1}_{system}_{DURATION}.pcap"
OUTPUT_FILE = f"{RAW_PATH}/{FILENAME}"

In [192]:
start_capture(INTERFACE, OUTPUT_FILE, DURATION)

Разрешен домен spotify.com -> ['2600:1901:1:7c5::', '35.186.224.24']
Разрешен домен open.spotify.com -> ['146.75.119.42', '2a04:4e42:8d::810']
Разрешен домен music.yandex.ru -> ['2a02:6b8::186', '213.180.204.186']
Разрешен домен mc.yandex.ru -> ['87.250.251.119', '2a02:6b8::1:119', '77.88.21.119', '87.250.250.119']
Разрешен домен music.apple.com -> ['2a02:26f0:e80:183::2a1', '2a02:26f0:e80:18e::2a1', '23.196.52.31', '2a02:26f0:e80:198::2a1', '2a02:26f0:e80:182::2a1', '2a02:26f0:e80:18f::2a1']
Разрешен домен zaycev.net -> ['176.58.34.241']
host 2a02:26f0:e80:183::2a1 or host 2600:1901:1:7c5:: or host 2a02:6b8::186 or host 146.75.119.42 or host 2a02:6b8::1:119 or host 2a02:26f0:e80:18e::2a1 or host 2a02:26f0:e80:18f::2a1 or host 213.180.204.186 or host 23.196.52.31 or host 87.250.250.119 or host 2a02:26f0:e80:198::2a1 or host 2a02:26f0:e80:182::2a1 or host 35.186.224.24 or host 87.250.251.119 or host 176.58.34.241 or host 77.88.21.119 or host 2a04:4e42:8d::810
BPF-фильтр: host 2a02:26f0: